In [352]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image

In [ ]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (5.0, 4.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

%load_ext autoreload
%autoreload 2


In [ ]:
def delete_zone_identifier(filenames, directory):
    for filename in filenames:
        try:
            name = filename.split(":")[1]
        except:
            name = ""
        if name == "Zone.Identifier":
            os.remove(directory + "/" + filename)

In [ ]:
filenames_train = os.listdir("train/train")
delete_zone_identifier(filenames_train, "train/train")
filenames_test = os.listdir("test1/test1")
delete_zone_identifier(filenames_test, "test1/test1")

In [ ]:
def preprocess_image(file_path, target_size):
    image = Image.open(file_path)
    image = image.convert('RGB')
    image = image.resize(target_size)
    image = np.array(image, dtype="float32") / 255.0
    return image

In [ ]:
def parse_images(filenames, directory, limit_dogs, limit_cats):
    for filename in filenames:
        category = filename.split('.')[0]
        if category == "dog":
            limit_dogs -= 1
            if limit_dogs < 0:
                os.remove(directory + "/" + filename)
        else:
            limit_cats -= 1
            if limit_cats < 0:
                os.remove(directory + "/" + filename)

In [ ]:
filenames_train = os.listdir("train/train")
parse_images(filenames_train, "train/train", 500, 500)
filenames_test = os.listdir("test1/test1")
parse_images(filenames_test, "test1/test1", 100, 100)

In [ ]:
def load_dataset(filenames):
    images = []
    labels = []
    for filename in filenames:
        image = preprocess_image("train/train/" + filename, (64, 64))
        images.append(image)
        category = filename.split('.')[0]
        if category == "dog":
            labels.append(1)
        else:
            labels.append(0)
    return np.array(images), np.array(labels)

In [ ]:
filenames = os.listdir("train/train")
train_set_x_orig, train_set_y = load_dataset(filenames_train)

In [ ]:
train_set_x = train_set_x_orig.reshape(train_set_x_orig.shape[0], -1).T
train_set_y = train_set_y.reshape(1, train_set_y.shape[0])

In [378]:
train_set_x.shape, train_set_y.shape

((12288, 1000), (1, 1000))

In [377]:
print(train_set_x, train_set_y)

[[0.1764706  0.7764706  0.1882353  ... 0.9607843  0.4627451  0.5176471 ]
 [0.16078432 0.78039217 0.1882353  ... 0.9647059  0.32941177 0.36078432]
 [0.15686275 0.75686276 0.1882353  ... 0.94509804 0.22352941 0.2627451 ]
 ...
 [0.49411765 0.4        0.7137255  ... 0.70980394 0.654902   0.5803922 ]
 [0.5176471  0.38039216 0.7137255  ... 1.         0.5803922  0.40784314]
 [0.5647059  0.3254902  0.7058824  ... 0.9882353  0.5529412  0.30980393]] [[1 0 0 0 0 0 0 1 1 1 1 0 0 0 1 1 1 1 1 1 0 0 1 1 0 1 0 1 1 1 1 0 1 1 0 0
  1 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 0 1 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1
  1 0 1 0 0 1 1 1 1 1 0 1 1 0 1 1 0 0 1 0 1 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1
  1 1 1 1 1 1 0 1 0 0 1 1 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 1 0 0 1 0 1 1 0 1
  1 0 0 0 1 1 0 0 1 1 1 1 1 0 1 0 0 1 0 0 1 1 0 0 0 0 1 0 1 0 1 1 1 1 1 0
  0 0 1 1 0 1 0 0 1 0 0 1 0 1 1 1 0 0 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 1 1 1
  0 1 0 1 1 1 1 0 0 0 1 0 0 0 0 0 0 1 1 0 0 1 0 0 1 0 0 0 0 1 1 1 0 0 0 0
  0 1 0 0 0 0 0 1 1 1 1 0 1 0 0 1 1 0 

In [ ]:
LAYER_DIMS = [12288, 12, 7, 5, 1]

In [ ]:
def initialize_parameters(layer_dims):
    np.random.seed(2)
    parameters = {}
    L = len(layer_dims)

    for l in range(1, L):
        parameters['W' + str(l)] = np.random.randn(layer_dims[l], layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l], 1))
        
    return parameters

In [ ]:
parameters = initialize_parameters(LAYER_DIMS)
print(parameters)

In [ ]:
def RELU(Z):
    cache = Z
    A = np.maximum(Z, 0)
    return A, cache

In [ ]:
def sigmoid(Z):
    cache = Z
    A = 1 / (1 + np.exp(-Z))
    return A, cache

In [ ]:
def linear_forward(A, W, b):
    Z = np.matmul(W, A) + b
    cache = (A, W, b)
    return Z, cache

In [ ]:
def linear_activation_forward(prev_A, W, b, activ_funct):
    if activ_funct == "RELU":
        Z, linear_cache = linear_forward(prev_A, W, b)
        A, activation_cache = RELU(Z)

    elif activ_funct == "sigmoid":
        Z, linear_cache = linear_forward(prev_A, W, b)
        A, activation_cache = sigmoid(Z)
    cache = (linear_cache, activation_cache)
    
    return A, cache

In [ ]:
def forward_propagation(X, parameters):
    caches = []
    L = len(parameters) //2
    A = X

    for l in range(1, L):
        A_prev = A
        A, cache = linear_activation_forward(A_prev, parameters['W'+str(l)], parameters['b'+str(l)], "RELU")
        caches.append(cache)
    
    AL, cache = linear_activation_forward(A, parameters['W'+str(L)], parameters['b'+str(L)], "sigmoid")
    caches.append(cache)

    return AL, caches

In [363]:
def cost_function(AL, Y):
    m = Y.shape[1]
    element1 = np.multiply(Y, np.log(AL))
    element2 = np.multiply((1 - Y), np.log(1 - AL))
    cost = (-1 / m) * np.sum(element1 + element2)

    cost = np.squeeze(cost)

    return cost

In [ ]:
def RELU_backward(dA, activ_cache):
    Z = activ_cache
    dZ = np.array(dA, copy=True)
    dZ = np.where(dZ <= 0, 0, dZ)
    
    return dZ

In [ ]:
def sigmoid_backward(dA, activ_cache):
    Z = activ_cache

    x = 1 / (1 + np.exp(-Z))
    dZ = dA * x * (1-x)

    return dZ

In [ ]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = (1 / m) * np.matmul(dZ, A_prev.T)
    db = (1 / m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.matmul(W.T, dZ)
    
    return dA_prev, dW, db

In [ ]:
def linear_activation_backward(dA, cache, activ_funct):
    linear_cache, activ_cache = cache
    if activ_funct == "relu":
        dZ = RELU_backward(dA, activ_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)
    elif activ_funct == "sigmoid":
        dZ = sigmoid_backward(dA, activ_cache)
        dA_prev, dW, db = linear_backward(dZ, linear_cache)

    return dA_prev, dW, db

In [369]:
def backward_propagation(AL, Y, caches):
    gradients = {}
    L = len(caches)
    Y = Y.reshape(AL.shape)
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    current_cache = caches[L-1]
    dA_prev, dW, db = linear_activation_backward(dAL, current_cache, "sigmoid")
    gradients['dA'+str(L-1)] = dA_prev
    gradients['dW'+str(L)] = dW
    gradients['db'+str(L)] = db

    for l in reversed(range(L-1)):
        current_cache = caches[l]
        dA_prev, dW, db = linear_activation_backward(gradients['dA'+str(l+1)], current_cache, "relu")
        gradients['dA'+str(l)] = dA_prev
        gradients['dW'+str(l+1)] = dW
        gradients['db'+str(l+1)] = db
    
    return gradients


In [ ]:
def update_parameters(parameters, gradients, learning_rate):
    L = len(parameters) // 2
    parameters = parameters.copy()
    for l in range(L):
        parameters['W'+str(l+1)] = parameters['W'+str(l+1)] - learning_rate * gradients['dW'+str(l+1)]
        parameters['b'+str(l+1)] = parameters['b'+str(l+1)] - learning_rate * gradients['db'+str(l+1)]
    return parameters

In [373]:
def DNN_model(X, Y, layer_dims, learning_rate, num_iterations, print_cost):
    
    np.random.seed(2)
    costs = []
    parameters = initialize_parameters(layer_dims)
    for i in range(0, num_iterations):
        AL, caches = forward_propagation(X, parameters)
        cost = cost_function(AL, Y)
        gradients = backward_propagation(AL, Y, caches)
        parameters = update_parameters(parameters, gradients, learning_rate)
        if print_cost and i % 100 == 0:
            print (f"Cost after iteration {i}: {cost}")
        if print_cost and i % 100 == 0:
            costs.append(cost)
    
    return parameters, costs

In [379]:
parameters, costs = DNN_model(train_set_x, train_set_y, LAYER_DIMS, learning_rate=0.001, num_iterations = 2500, print_cost = True)

Cost after iteration 0: 0.6931471980329036
Cost after iteration 100: 0.6931471827337273
Cost after iteration 200: 0.6931471772290387
Cost after iteration 300: 0.6931471765974055
Cost after iteration 400: 0.6931471768337675
Cost after iteration 500: 0.6931471774651263
Cost after iteration 600: 0.6931471778878723
Cost after iteration 700: 0.6931471784018445
Cost after iteration 800: 0.6931471790277555
Cost after iteration 900: 0.6931471799002499
Cost after iteration 1000: 0.6931471804728976
Cost after iteration 1100: 0.6931471805190826
Cost after iteration 1200: 0.6931471805554509
Cost after iteration 1300: 0.6931471806406277
Cost after iteration 1400: 0.6931471806425105
Cost after iteration 1500: 0.6931471806199584
Cost after iteration 1600: 0.6931471805868971
Cost after iteration 1700: 0.6931471805599454


KeyboardInterrupt: 